 Loading data

In [1]:
# Import libraries
import os
from PIL import Image
from tqdm import tqdm
import numpy as np

def load_data(data_dir, paths_only=False):
    """
    Either loads images as numpy arrays or the paths.
    data_dir is expected to contain data split by class in subfolders
    """

    # Read classes
    classes = sorted(os.listdir(data_dir))

    # Initialize train and test data list
    train_images, test_images = [], []
    train_labels, test_labels = [], []

    # Set training data ratio
    train_ratio = 0.7

    for idx, class_name in enumerate(tqdm(classes)):

        # Read images in class folder
        class_data = os.listdir(os.path.join(data_dir, class_name))

        # Train data are the first images in class-folder
        for img_name in class_data[:int(train_ratio*len(class_data))]:

            if paths_only: # Get only images paths
                train_images.append(os.path.join(data_dir, class_name, img_name))

            else: # Load data and append it to train_images list
                image = Image.open(os.path.join(data_dir, class_name, img_name))
                train_images.append(np.array(image))

        # Identify every image in the flattened list train_images with its class
        train_labels += [idx]*int(train_ratio*len(class_data))

        # Test data are the last images
        for img_name in class_data[int(train_ratio*len(class_data)):]:

            if paths_only: # Get only images paths
                test_images.append(os.path.join(data_dir, class_name, img_name))

            else: # Load data and append it to test_images list
                image = Image.open(os.path.join(data_dir, class_name, img_name))
                test_images.append(np.array(image))

        # Identify every image in the flattened list test_images with its class
        test_labels += [idx]*(len(class_data) - int(train_ratio*len(class_data)))

    if paths_only: # Return it as numpy arrays
        return (classes,
                np.array(train_images), np.array(train_labels),
                np.array(test_images), np.array(test_labels))
    else: # Concatenate images into a single float numpy array of shape (#imgs, width, height, channels)
        train_images = np.stack(train_images).astype(float)
        train_labels = np.array(train_labels)
        test_images = np.stack(test_images).astype(float)
        test_labels = np.array(test_labels)
        return classes, train_images, train_labels, test_images, test_labels

VGG 

In [2]:
import torch
import argparse

from tqdm import tqdm

import torchvision.models as models

from PIL import Image
from torchvision import transforms
from torchvision.models import VGG16_Weights

# Initialing compute device (use GPU if available).
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

normalization_std = [0.229, 0.224, 0.225]
normalization_mean = [0.485, 0.456, 0.406]

test_transforms = transforms.Compose([transforms.ToTensor(),
                              transforms.RandomResizedCrop(224),
                              transforms.Normalize(mean=normalization_mean,
                                                   std=normalization_std)])

classes, train_images, train_labels, test_images, test_labels = \
    load_data("Data", True)

# Utility function.
def image_loader(image_name):
    image = Image.open(image_name).convert('RGB')
    # Fake batch dimension required to fit network's input dimensions.
    image = test_transforms(image).unsqueeze(0)

    return image.to(device)

model = models.vgg16(weights=VGG16_Weights).features.to(device)
model.eval()


red_train_img = []
for image_path, label in tqdm(zip(train_images, train_labels)):
    image = image_loader(image_path)
    features = model(image)
    red_train_img.append(features.data.detach().cpu().numpy().flatten())

red_test_img = []
for image_path, label in tqdm(zip(test_images, test_labels)):
    image = image_loader(image_path)
    features = model(image)
    red_test_img.append(features.data.detach().cpu().numpy().flatten())



100%|██████████| 4/4 [00:00<00:00, 251.11it/s]
c:\Users\evida\miniconda3\envs\test\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
2165it [08:55,  4.04it/s]
931it [05:24,  2.87it/s]


In [3]:
# Check vgg is working correctly

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, adjusted_mutual_info_score

km = KMeans(n_clusters=len(classes))

clusters = km.fit_predict(red_train_img)

ari = adjusted_rand_score(train_labels, clusters)
nmi = normalized_mutual_info_score(train_labels, clusters)
ami = adjusted_mutual_info_score(train_labels, clusters)

print(ari,nmi,ami)


c:\Users\evida\miniconda3\envs\test\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.
  warnings.warn(


0.0064788353151242145 0.024406183086212077 0.02273851243901476


In [24]:
# Save reduced data

data2save = {
    "classes": classes,
    "train_data": red_train_img,
    "train_labels": train_labels,
    "test_data": red_test_img,
    "test_labels": test_labels
}
import pickle

with open("here.pickle", "wb") as f:
    pickle.dump(data2save,f)
    f.close()


b [0 0 0 ... 3 3 3]


In [8]:
#Open reduced data

import pickle

with open("here.pickle", "rb") as f:
    b = pickle.load(f)
    f.close()

classes=b["classes"]
train_images=b["train_data"]
train_labels=b["train_labels"]
test_images=b["test_data"]
test_labels=b["test_labels"]


np_array = np.array(train_images)
print("Shape of the list:", np_array.shape)


Shape of the list: (2165, 25088)


Linear kernel

In [9]:

import numpy as np

from sklearn.svm import SVC
from sklearn import datasets
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score


# Initialize parameters.
kernel = 'linear'
max_iteration = 1000

# Initialise an SVM classification model for each one of the 4 classes.
models = [SVC(kernel=kernel, max_iter=max_iteration, probability=True) for i in range (4)]

# Train the models.
for i in range(4):
  models[i].fit(train_images, train_labels==i)

c:\Users\evida\miniconda3\envs\test\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\evida\miniconda3\envs\test\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\evida\miniconda3\envs\test\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\evida\miniconda3\envs\test\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [7]:

predicted_scores = []
for i in range(4):
    predicted_scores.append(models[i].predict_proba(test_images)[:,1])

predicted_scores = np.asarray(predicted_scores)
predicted = np.argmax(predicted_scores,axis=0)

conf_mat = confusion_matrix(test_labels, predicted)
print(conf_mat)

print("Accuracy =", accuracy_score(test_labels,predicted))
print("Precision =", precision_score(test_labels,predicted,average='macro'))
print("Recall =", recall_score(test_labels,predicted,average='macro'))

[[ 23 194   0  54]
 [ 12 206   0  56]
 [ 11 104   0  17]
 [ 16 177   0  61]]


Results:

[[199  45   5  22]

[ 70 149  10  45]

[ 12  24  86  10]

[ 12  35   5 202]]

Accuracy = 0.6831364124597207

Precision = 0.7008621963326671

Recall = 0.6812259264194301

rbf kernel

In [ ]:
import numpy as np

from sklearn.svm import SVC
from sklearn import datasets
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score


# Initialize parameters.
kernel = 'rbf'
max_iteration = 1000

# Initialise an SVM classification model for each one of the 4 classes.
models = [SVC(kernel=kernel, max_iter=max_iteration, probability=True) for i in range (4)]

# Train the models.
for i in range(4):
  models[i].fit(train_images, train_labels==i)

predicted_scores = []
for i in range(4):
    predicted_scores.append(models[i].predict_proba(test_images)[:,1])

predicted_scores = np.asarray(predicted_scores)
predicted = np.argmax(predicted_scores,axis=0)

conf_mat = confusion_matrix(test_labels, predicted)
print(conf_mat)

print("Accuracy =", accuracy_score(test_labels,predicted))
print("Precision =", precision_score(test_labels,predicted,average='macro'))
print("Recall =", recall_score(test_labels,predicted,average='macro'))

Results:

[[205  46   3  17]

 [ 74 146   6  48]

 [ 16  25  80  11]

 [ 11  33   2 208]]

Accuracy = 0.686358754027927

Precision = 0.7163624714602345

Recall = 0.6785656309399986